In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

from Utils import exec_jupyter
exec_jupyter()

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook, tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
np.set_printoptions(suppress=True)

In [ ]:
from DataGenBeer import DataGen
dg = DataGen()

In [ ]:
import model.CNN_beer_sent as CNN
Model = CNN.Model

In [ ]:
vocab_size = dg.vecs.vocab_size
embed_size = dg.vecs.word_dim

In [ ]:
model = Model(vocab_size, embed_size, 32, dirname='beer', pre_embed=dg.vecs.embeddings)

In [ ]:
X, y = dg.get_training_data('overall')

In [ ]:
Xt, yt = dg.get_test_data('overall')

In [ ]:
def train(model, name) :
    for i in tqdm_notebook(range(4)) :
        if name.startswith('normal') :
            correct=False
        else :
            if i >= 1 :
                correct=True
            else :
                correct=False
        loss = model.train(X, y, correct=correct)
        print(loss)
        o, he = model.evaluate(Xt)
        o = np.array(o)
        rep = classification_report(yt, (o > 0.5))
        print(rep)
        stmt = '%s, %s' % (i, loss)
        dirname = model.save_values(add_name=name)
        f = open(dirname + '/epoch.txt', 'a')
        f.write(stmt + '\n')
        f.write(rep + '\n')
        f.close()

In [ ]:
model.load_values('outputs/attnexp_cnn_beer_MonMay2122:17:322018_kld_overall/')

In [ ]:
model.load_values('outputs/attnexp_cnn_beer_MonMay2117:27:572018_normal_overall/')

In [ ]:
train(model, name='kld_overall')

In [ ]:
def load_model(dirname) :
    model = Model(vocab_size, embed_size, 32, dirname='beer', pre_embed=dg.vecs.embeddings)
    model.dirname = dirname
    model.load_values(dirname)
    return model

In [ ]:
dirname_normal = 'outputs/attnexp_cnn_beer_MonMay2122:47:392018_normal_overall/'
model_normal = load_model(dirname_normal)
dirname_kld = 'outputs/attnexp_cnn_beer_MonMay2122:57:232018_kld_overall/'
model_kld = load_model(dirname_kld)

Evaluation
==========

In [ ]:
import matplotlib.ticker as ticker

def showAttention(input_sentence, attentions):
    # Set up figure with colorbar
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    cax = ax.imshow(np.expand_dims(attentions, 0).T, cmap='bone', aspect='auto', vmin=0)
    fig.colorbar(cax)

    # Set up axes
    ax.set_yticks(np.arange(len(input_sentence)))
    ax.set_yticklabels(input_sentence)

    plt.show()

In [ ]:
np.random.seed(1230)
test_idx = np.random.choice(range(len(Xt)), size=2000, replace=False)
Xtest = [Xt[i] for i in test_idx]
ytest = [yt[i] for i in test_idx]

In [ ]:
import pickle
def generate_sampled(model) :
    attn_list = []
    sampled_list = []
    for i in tqdm_notebook(range(10)) :
        _, attn, sampled = model.evaluate(Xtest, sample=True)
        attn_list.append(attn)
        sampled_list.append(sampled)
        
    pickle.dump([attn_list, sampled_list], open(model.dirname + '/sampled_1.p', 'wb'))
        
def load_sampled(model) :
    attn_list, sampled_list = pickle.load(open(model.dirname + '/sampled_1.p', 'rb'))
    attn = list(map(list, zip(*attn_list)))
    attn = [np.array(x) for x in attn]
    attn = [x.mean(0) for x in attn]

    sampled = list(map(list, zip(*sampled_list)))
    sampled = [np.array(x) for x in sampled]
    sampled = [x.mean(0) for x in sampled]

    model.attn = attn
    model.sampled = sampled

In [ ]:
generate_sampled(model_normal)
generate_sampled(model_kld)

In [ ]:
import pickle

In [ ]:
load_sampled(model_normal)
load_sampled(model_kld)

In [ ]:
from scipy.stats import spearmanr, kendalltau
from cycler import cycler
import matplotlib.pyplot as plt

plt.rc('lines', linewidth=2)
line_cycle = cycler('linestyle', ['-', '--'])
color_cycle = cycler('color', ['#1f78b4', '#33a02c'])
marker_cycle = cycler('marker', ['+', '2'])

def plot(attn, sampled, ax, name, marker) :
    a = []
    b = []
    for i in range(len(attn)) :
        for j in range(len(attn[i])) :
            v = attn[i][j]
            if v > 0.0 :
                b += [sampled[i][j]]
                a += [v]

    a = np.array(a)
    b = np.array(b)
    print(spearmanr(a, b))
    print(kendalltau(a, b))
    print(np.corrcoef(a, b))
    fit = np.polyfit(a, b, deg=1)
    r = np.linspace(0, 1, 10)
    ax.plot(r, fit[0] * r + fit[1], label=name + (r' ($\rho$=%.3f'%np.corrcoef(a, b)[0, 1]) + ')')
    ax.scatter(a, b, s=0.8, alpha=0.5, marker=marker)
    ax.set_xlabel('Attention')
    ax.set_ylabel('Expected Normalised KL divergence')

In [ ]:
fig, ax = plt.subplots()
ax.set_prop_cycle(line_cycle + color_cycle) # + marker_cycle)
plot(model_normal.attn, model_normal.sampled, ax, name='Standard', marker='^')
plot(model_kld.attn, model_kld.sampled, ax, name='Regularized', marker='*')
ax.legend()
plt.tight_layout()

In [ ]:
plt.hist(a)

In [ ]:
otest, hetest = model_normal.evaluate(Xtest)

In [ ]:
def showAttention(fig, ax, input_sentence, attnn, kldn, attnk, kldk):

    attentions = np.stack([attnn - kldn, attnk - kldk], axis=0)[:, :len(input_sentence)]
    
    im = ax.imshow(attentions.T, cmap='bwr', interpolation='none', vmin=-1, vmax=1)
    cax = fig.colorbar(im)
    
    # Set up axes
    ax.set_xticks([])
    ax.set_xticklabels([])
    
    ax.set_yticks(np.arange(len(input_sentence)))
    ax.set_yticklabels(input_sentence, fontsize=20)
    ax.tick_params(axis="y",direction="in")
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")

In [ ]:
idxs = []
for idx, (att, sm) in enumerate(zip(model_normal.attn, model_normal.sampled)) :
    kld = (att * np.log((att + 1e-8)/(sm + 1e-8))).sum()
    if kld != kld :
        continue
    if kld > 1. :
        idxs.append(idx)

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("output_beer.pdf")
for i in idxs :
    n = i
    words = [" ".join([dg.vecs.idx2word[w] for w in x]) for x in Xtest[n]]   
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))        
    showAttention(fig, axes, words, model_normal.attn[n], model_normal.sampled[n], model_kld.attn[n], model_kld.sampled[n])
    pdf.savefig(fig, bbox_inches='tight')
pdf.close()